In [ ]:
def strategy_5(data):
    n = 26
    
    data['ma_5'] = data['close'].rolling(window=10).mean()
    data['Change_5'] = abs(data['close'] - data['close'].shift(10))
    data['Volatility_5'] = abs(data['close'] - data['close'].shift()).rolling(n).sum()
    data['ER_5'] = data['Change_5'] / data['Volatility_5']
    data['SC_5'] = np.square(data['ER_5'] * (2.0 / (2 + 1) - 2.0 / (30 + 1)) + 2.0 / (30 + 1))
    
    data['KAMA_5'] = data['close'].rolling(n).mean()
    data['KAMA_5'][:n] = np.nan

    i = 1
    while i < len(data['KAMA_5'][n+1:]):
        s = data['KAMA_5']
        s.iloc[n + i] = data['KAMA_5'][n + i - 1] + data['SC_5'][n + i] * (data['close'][n + i] - data['KAMA_5'][n + i - 1])
        data['KAMA_5'] = s
        i += 1

    data = data.drop(['Change_5', 'Volatility_5', 'ER_5', 'SC_5'], axis=1)
    data['MACD_KAMA_5'] = data['KAMA_5'].ewm(span=12, adjust=False).mean() - data['KAMA_5'].ewm(span=26, adjust=False).mean()
    data['signal_line_5'] = data['MACD_KAMA_5'].rolling(window=9).mean()

    def rsi(data, period=14):
        data['rsi_5'] = ta.RSI(data['close'], period)
        return data
    
    data = rsi(data)
    data['signals'] = 0
    flag = 0
    for i in range(len(data)):
        if data.MACD_KAMA_5[i] > data.signal_line_5[i] and data.rsi_5[i] > 70:
            data.signals[i] = 1
            flag = 1
        elif data.MACD_KAMA_5[i] < data.signal_line_5[i] and data.rsi_5[i] < 30:
            data.signals[i] = -1
            flag = 0

    return data
